In [1]:
### Inference on a single image
import os
import json
import torch
import torch.nn as nn
from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from PIL import Image

# --- Configuration ---
MODEL_PATH = 'models/best_model.pth'
# IMAGE_PATH = r'dataset\images\test\Hummus\18.jpg'  # <<<--- Change this to your image file
IMAGE_PATH = r"Test_data/Balaleet/train_29.jpg"
MAPPING_JSON_PATH = 'class_mapping.json'
NUM_CLASSES = 66

# --- Device Setup ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Load Class Mapping ---
print(f"Loading class mapping from: {MAPPING_JSON_PATH}")
try:
    with open(MAPPING_JSON_PATH, 'r') as f:
        class_to_idx = json.load(f)
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    if len(idx_to_class) != NUM_CLASSES:
        print(f"Warning: Found {len(idx_to_class)} classes, expected {NUM_CLASSES}")
except Exception as e:
    raise RuntimeError(f"Failed to load class mapping: {e}")

# --- Load Model ---
print("Loading model...")
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
model = efficientnet_v2_s(weights=weights)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.37, inplace=True), # Keep dropout
    nn.Linear(num_ftrs, NUM_CLASSES),
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# --- Preprocessing ---
preprocess = weights.transforms()

# --- Prediction Function ---
def predict_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        input_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = model(input_tensor)
            conf, idx = torch.max(torch.softmax(output, dim=1), 1)
        return idx_to_class.get(idx.item(), "Unknown"), conf.item()
    except Exception as e:
        print(f"Prediction error: {e}")
        return None, None

# --- Inference ---
if not os.path.exists(IMAGE_PATH):
    print(f"Image path does not exist: {IMAGE_PATH}")
else:
    print(f"Predicting image: {IMAGE_PATH}")
    label, confidence = predict_image(IMAGE_PATH)
    if label:
        print(f"-> Predicted Class: {label}")
        print(f"-> Confidence: {confidence:.4f}")

Using device: cpu
Loading class mapping from: class_mapping.json
Loading model...
Predicting image: Test_data/Balaleet/train_29.jpg
-> Predicted Class: Balaleet
-> Confidence: 0.9967


In [1]:
import os
import json
import csv
import torch
import torch.nn as nn
from PIL import Image
from pathlib import Path
from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s

# --- Configuration ---
MODEL_PATH = 'models/best_model.pth'
MAPPING_JSON_PATH = 'class_mapping.json'
NUM_CLASSES = 66
TEST_ROOT_DIR = 'Test_data'  # Root directory with subfolders of class names

# --- Device Setup ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Load Class Mapping ---
print(f"Loading class mapping from: {MAPPING_JSON_PATH}")
try:
    with open(MAPPING_JSON_PATH, 'r') as f:
        class_to_idx = json.load(f)
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    if len(idx_to_class) != NUM_CLASSES:
        print(f"Warning: Found {len(idx_to_class)} classes, expected {NUM_CLASSES}")
except Exception as e:
    raise RuntimeError(f"Failed to load class mapping: {e}")

# --- Load Model ---
print("Loading model...")
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
model = efficientnet_v2_s(weights=weights)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.37, inplace=True),
    nn.Linear(num_ftrs, NUM_CLASSES),
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# --- Preprocessing ---
preprocess = weights.transforms()

# --- Prediction Function ---
def predict_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        input_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = model(input_tensor)
            conf, idx = torch.max(torch.softmax(output, dim=1), 1)
        return idx_to_class.get(idx.item(), "Unknown"), conf.item()
    except Exception as e:
        print(f"Prediction error for {image_path}: {e}")
        return None, None

# --- Gather All Images ---
def get_all_image_paths(root_dir):
    return list(Path(root_dir).rglob("*.jpg")) + \
           list(Path(root_dir).rglob("*.jpeg")) + \
           list(Path(root_dir).rglob("*.png"))

all_images = get_all_image_paths(TEST_ROOT_DIR)

# --- Run Inference ---
if not all_images:
    print(f"No images found in: {TEST_ROOT_DIR}")
else:
    print(f"Found {len(all_images)} images. Running inference...\n")

    results = []
    for image_path in all_images:
        label, confidence = predict_image(str(image_path))
        results.append((str(image_path), label, confidence))
        print(f"Image: {image_path}")
        print(f"-> Predicted Class: {label}")
        print(f"-> Confidence: {confidence:.4f}\n")

    # --- Save Results to CSV ---
    output_csv = "predictions.csv"
    with open(output_csv, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Image Path", "Predicted Class", "Confidence"])
        writer.writerows(results)

    print(f"Predictions saved to: {output_csv}")


Using device: cpu
Loading class mapping from: class_mapping.json
Loading model...
Found 269 images. Running inference...

Image: Test_data/Shrimp/train_74.jpg
-> Predicted Class: Shrimp
-> Confidence: 0.9969

Image: Test_data/Shrimp/train_65.jpg
-> Predicted Class: Shrimp
-> Confidence: 0.9907

Image: Test_data/Shrimp/train_104.jpg
-> Predicted Class: Shrimp
-> Confidence: 0.9850

Image: Test_data/Shrimp/train_103.jpg
-> Predicted Class: Shrimp
-> Confidence: 0.9760

Image: Test_data/Orange (fruit)/train_20.jpg
-> Predicted Class: Orange (fruit)
-> Confidence: 0.9954

Image: Test_data/Orange (fruit)/test_652acb4266a31500.jpg
-> Predicted Class: Orange (fruit)
-> Confidence: 0.7617

Image: Test_data/Orange (fruit)/train_70.jpg
-> Predicted Class: Orange (fruit)
-> Confidence: 0.9807

Image: Test_data/Orange (fruit)/test_0697d514e448855d.jpg
-> Predicted Class: Orange (fruit)
-> Confidence: 0.9928

Image: Test_data/Pomegranate/train_72.jpg
-> Predicted Class: Pomegranate
-> Confidence: 0